In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from geopy.distance import geodesic
warnings.filterwarnings("ignore")
%cd D:\Resources\Datasets\深圳北站周边交通拥堵指数

D:\Resources\Datasets\深圳北站周边交通拥堵指数


In [2]:
# import moxing as mox
# mox.file.copy_parallel('obs://yxkml/traffic', '.')

In [ ]:
lands = np.array([[(22.587644,114.018583), (22.588694,114.021737), (22.589625,114.024377), (22.590536,114.026844)], 
                  [(22.589526,114.015493), (22.593686,114.015987), (22.597192,114.016480), (22.601392,114.016652)], 
                  [(22.605255,114.018604), (22.608999,114.021308), (22.610029,114.023947), (22.610920,114.026329)], 
                  [(22.594082,114.029956), (22.597688,114.031501), (22.600719,114.029698), (22.603690,114.027209)], 
                  [(22.602601,114.036436), (22.607434,114.032359), (22.611673,114.028582), (22.614525,114.024720)], 
                  [(22.608465,114.025595), (22.606305,114.027290), (22.604661,114.028664), (22.603096,114.030080)]])
directions = np.array([90, 270, 0, 180, 90, 270, 0, 180, 0, 180, 180, 0])
id_roads = np.array([276183, 276184, 275911, 275912, 276240, 276241, 
                     276264, 276265, 276268, 276269, 276737, 276738])
times = pd.date_range('2018-12-31 00:00:00', '2019-04-01 00:00:00', freq='10T')
times = pd.date_range('2019-9-30 00:00:00', '2019-12-01 00:00:00', freq='10T')
times = pd.date_range('2019-11-30 00:00:00', '2019-12-22 00:00:00', freq='10T')
times = pd.date_range('2019-12-20 00:00:00', '2020-01-02 00:00:00', freq='10T')
total = pd.DataFrame(0, index=id_roads, columns=times)
count = pd.DataFrame(0, index=id_roads, columns=times)

In [ ]:
fp = open('D:/Resources/Datasets/深圳北站周边交通拥堵指数/toPredict_train_gps.csv')
c = 0
dis = np.zeros((6, 4))
while True:
    lst = fp.readlines(1024*1024*1024)
    c = c + 1
    print(c, 'G')
    if not lst:
        break
    for s in lst:
        s = s[s.find('[')+1:s.find(']')]
        for array in s.split(', '):
            array = eval(array.replace(' ', ','))
            for i in range(6):
                for j in range(4):
                    dis[i, j] = geodesic((array[1], array[0]), lands[i ,j]).m
            i = dis.min(1).argmin()
            i = 2*i+np.abs(directions[2*i:2*i+2] - array[3]).argmin()
            t = pd.Timestamp(array[4], unit='s').floor('10T')
            total.loc[id_roads[i], t] += array[2]
            count.loc[id_roads[i], t] += 1
fp.close()

In [ ]:
df = total / count
df.to_csv('toPredict_train_speed.csv')
count.to_csv('toPredict_train_count.csv')

In [3]:
# 合并三个GPS的speed
df1 = pd.read_csv('201901_201903_speed.csv', index_col='id_road')
df2 = pd.read_csv('201910_11_speed.csv', index_col='id_road')
df3 = pd.read_csv('20191201_20191220_speed.csv', index_col='id_road')
df1.fillna(method='ffill', inplace=True, axis=1)
df2.fillna(method='ffill', inplace=True, axis=1)
df3.fillna(method='ffill', inplace=True, axis=1)
df4 = df1.iloc[:, 144:-1].join(df2.iloc[:, 144:-1]).join(df3.iloc[:, 144:-1])
df4.columns = pd.to_datetime(df4.columns)

In [4]:
# 合并三个GPS的count
df1 = pd.read_csv('201901_201903_count.csv', index_col='id_road')
df2 = pd.read_csv('201910_11_count.csv', index_col='id_road')
df3 = pd.read_csv('20191201_20191220_count.csv', index_col='id_road')
df1.replace(0, np.nan, inplace=True)
df2.replace(0, np.nan, inplace=True)
df3.replace(0, np.nan, inplace=True)
df1 = df1.fillna(method='ffill', axis=1)
df2 = df2.fillna(method='ffill', axis=1)
df3 = df3.fillna(method='ffill', axis=1)
df5 = df1.iloc[:, 144:-1].join(df2.iloc[:, 144:-1]).join(df3.iloc[:, 144:-1])
df5.columns = pd.to_datetime(df5.columns)

In [5]:
# 加入carspeed和count
df6 = pd.read_csv('train_TTI_fillna.csv')
df6['time'] = pd.to_datetime(df6['time'])
df6['carspeed'] = 0
df6['count'] = 0
for i in df6.index:
    df6.loc[i, 'carspeed'] = df4.loc[df6.loc[i, 'id_road'], df6.loc[i, 'time']]
    df6.loc[i, 'count'] = df5.loc[df6.loc[i, 'id_road'], df6.loc[i, 'time']]

In [6]:
# df6.to_csv('train_TTI_fillna_carspeed_count.csv', index=False)

In [7]:
df6.corr()

,id_road,TTI,speed,carspeed,count
id_road,1.000000,0.263261,-0.677130,0.004982,0.035215
TTI,0.263261,1.000000,-0.592766,0.002093,-0.046711
speed,-0.677130,-0.592766,1.000000,-0.003638,-0.052217
carspeed,0.004982,0.002093,-0.003638,1.000000,-0.002583
count,0.035215,-0.046711,-0.052217,-0.002583,1.000000


In [8]:
# 加入carspeed和count
df7 = pd.read_csv('toPredict_train_speed.csv', index_col='id_road')
df7.fillna(method='ffill', inplace=True, axis=1)
df7.columns = pd.to_datetime(df7.columns)
df8 = pd.read_csv('toPredict_train_count.csv', index_col='id_road')
df8.replace(0, np.nan, inplace=True)
df8 = df8.fillna(method='ffill', axis=1)
df8.columns = pd.to_datetime(df8.columns)
df9 = pd.read_csv('toPredict_train_TTI.csv')
df9['time'] = pd.to_datetime(df9['time'])
df9['carspeed'] = 0
df9['count'] = 0
for i in df9.index:
    df9.loc[i, 'carspeed'] = df7.loc[df9.loc[i, 'id_road'], df9.loc[i, 'time']]
    df9.loc[i, 'carspeed'] = df8.loc[df9.loc[i, 'id_road'], df9.loc[i, 'time']]
# df9.to_csv('toPredict_train_TTI_carspeed_count.csv', index=False)